In [14]:
import requests
import pandas as pd
from collections import defaultdict

In [2]:
human_genes = pd.read_csv('hmtg/human_MTG_2018-06-14_genes-rows.csv')
entrez_ids = human_genes['entrez_id']

In [15]:
def clean_result(r):
    return { 'entrez_id': r['entrez_id'],             
             'tags': set([ c['name'] for c in r['gene_classifications'] ]),
             'categories': set([ c['category'] for c in r['gene_classifications']]) }

In [19]:
block_size = 25

tags = defaultdict(set)
categories = defaultdict(set)

nids = len(entrez_ids)
for i in range(0, nids, block_size):
    block_ids = entrez_ids[i:i+block_size]
    block_id_str = ",".join(block_ids.apply(str).values)
    
    query = f"http://api.brain-map.org/api/v2/data/Gene/query.json?criteria=[entrez_id$in{block_id_str}]&include=gene_classifications"
    
    res = requests.get(query)
    for r in res.json()['msg']:
        cr = clean_result(r)
        tags[cr['entrez_id']].update(cr['tags'])
        categories[cr['entrez_id']].update(cr['categories'])
    
    if i % 1000 == 0:       
        print(f"{i+1} / {nids}")    

1 / 50281
1001 / 50281
2001 / 50281
3001 / 50281
4001 / 50281
5001 / 50281
6001 / 50281
7001 / 50281
8001 / 50281
9001 / 50281
10001 / 50281
11001 / 50281
12001 / 50281
13001 / 50281
14001 / 50281
15001 / 50281
16001 / 50281
17001 / 50281
18001 / 50281
19001 / 50281
20001 / 50281
21001 / 50281
22001 / 50281
23001 / 50281
24001 / 50281
25001 / 50281
26001 / 50281
27001 / 50281
28001 / 50281
29001 / 50281
30001 / 50281
31001 / 50281
32001 / 50281
33001 / 50281
34001 / 50281
35001 / 50281
36001 / 50281
37001 / 50281
38001 / 50281
39001 / 50281
40001 / 50281
41001 / 50281
42001 / 50281
43001 / 50281
44001 / 50281
45001 / 50281
46001 / 50281
47001 / 50281
48001 / 50281
49001 / 50281
50001 / 50281


In [24]:
res_df = pd.DataFrame({
    'entrez_id': entrez_ids,
    'tags': [ ';'.join(tags[eid]) for eid in entrez_ids ],
    'categories': [ ';'.join(categories[eid]) for eid in entrez_ids ],
})

In [26]:
res_df.to_csv('gene_tags.csv', index=False)